<a href="https://colab.research.google.com/github/Mschaid/nma_dl_project/blob/main/notebooks/nma_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import torch

from IPython.display import Image

# Background

**Revealing the multidimensional mental representations of natural objects underlying human similarity judgements**: [link](https://www.nature.com/articles/s41562-020-00951-3.epdf?sharing_token=nHVvIjL4RlwottzTvjgItNRgN0jAjWel9jnR3ZoTv0M2M6rn_U7HFAne1ZfilQzEBRHexV_lH_PgbXIY9yiNp7yXnn3qOHdywYiFVOWZQIy_pJjjNv1vFLYrZ6nmAuWmC3u_Qh81SlZMMBe4PI2qBw31vRsjHdk2XJntcS9fkMc%3D )

**Human alignment of neural network representations**: [link](https://arxiv.org/abs/2211.01201)

**What can 1.8 billion regressions tell us about the pressures shaping high-level visual representation in brains and machines?**: [link](https://www.biorxiv.org/content/10.1101/2022.03.28.485868v2)

# Research questions
- Do different kinds of neural network architectures better align with human behavioral representations? (e.g., [2])
- Need better control for some models (see [3])
- Can we improve the alignment between network and human representations? (i.e., learn a simple - linear transformation W as in [2])?
- Can we replace W with a simple non-linear transformation (e.g., small NN) to further improve alignment?


## Data resources: 
### [thing](https://osf.io/jum2f/)

### [odd one out](https://osf.io/f5rn6/)


In [10]:
Image(url = "https://github.com/Mschaid/nma_dl_project/blob/main/images/Screenshot%202023-07-13%20at%208.40.25%20AM.jpg?raw=true")

From the second to last layer (negotiable) we et the 1xn (image x number of neurons) and contrauct an imagex neurons matrix

Calculate the distance between between images in representational space 
	assume the closer the images the more correlated they are

we label the decision correct or wrong based on what humans chose 

The goal is to find a W ( in the image below) that complies with human decision
note on the image : while visually 3 seems furter away 2 is circled(squared) just for the sake of example
